In [1]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.8 MB/s 
     |████████████████████████████████| 7.6 MB 15.6 MB/s 
     |████████████████████████████████| 182 kB 41.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.9 MB/s 


In [15]:
import nltk

In [3]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [17]:
with open("/content/MyDrive/MyDrive/MyDrive/Harry.txt", "r") as input:
    input_ = input.read().strip()

a = nltk.tokenize.sent_tokenize(input_)
# result = a.replace('\u200c', '')
l = []
for i in a:
  i = i.replace('\n', '')
  l.append(i)

In [18]:
l

['Mr and Mrs Dursley, of number four, Privet Drive, were proud tosay that they were perfectly normal, thank you very much.',
 'Theywere the last people you’d expect to be involved in anythingstrange or mysterious, because they just didn’t hold with suchnonsense.',
 'Mr Dursley was the director of a firm called Grunnings, whichmade drills.',
 'He was a big, beefy man with hardly any neck,although he did have a very large moustache.',
 'Mrs Dursley wasthin and blonde and had nearly twice the usual amount of neck,which came in very useful as she spent so much of her time craningover garden fences, spying on the neighbours.',
 'The Dursleys had asmall son called Dudley and in their opinion there was no finerboy anywhere.',
 'The Dursleys had everything they wanted, but they also had asecret, and their greatest fear was that somebody would discoverit.',
 'They didn’t think they could bear it if anyone found out aboutthe Potters.',
 'Mrs Potter was Mrs Dursley’s sister, but they hadn’tmet fo

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoints = "facebook/bart-large-xsum"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [9]:
tokenizer.model_max_length

1024

In [16]:
nltk.download('punkt')
# sentences = nltk.tokenize.sent_tokenize(input_)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
max([len(tokenizer.tokenize(sentence)) for sentence in l])

103

In [22]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in l:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(l) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

7

In [23]:
[len(tokenizer.tokenize(c)) for c in chunks]

[1020, 1005, 1011, 1018, 971, 982, 812]

In [24]:
[len(tokenizer(c).input_ids) for c in chunks]

[1022, 1007, 1013, 1020, 973, 984, 814]

In [25]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [26]:
summary = []
for input in inputs:
  output = model.generate(**input)
  a = tokenizer.decode(*output, skip_special_tokens=True)
  summary.append(a)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 62 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [27]:
print(summary)

['The Dursley family, who lived in the same street as the Potters, were the victims of a series of strange and mysterious events that took place in the early hours of Tuesday morning.', 'He didn’t see the owlsswooping past in broad daylight, though people down in thestreet did; they pointed and gazed open-mouthed as owl after owlsped overhead.', 'The newsreader said: ‘It’s been a strange day for owls in the UK.', 'A man had just arrived in the street where the Dursley family lived.', 'Albus Dumbledore and Professor McGonagall were sitting on a cold hard wall, looking out of the window of the Great Hall, when the professor said, ‘You-Know-Who has gone.', 'Dumbledore and Professor McGonagall were sitting on the wall of the Great Hall, looking out of the window, when Dumbledore suddenly said, ‘Harry Potter should come and live with his aunt and uncle.', 'Albus Dumbledore and Professor McGonagall and Hagrid were standing on the steps of the Dursley house in Privet Drive, looking at Harry P

In [28]:
from datasets import load_dataset